In [1]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import datetime

import boto3
import sagemaker
from sagemaker import get_execution_role

In [2]:
with_categories = False
if with_categories:
    base_job_name = 'deepar-bikerental-with-categories'
else:
    base_job_name = 'deepar-bikerental-no-categories'

In [3]:
# Specify your bucket name
bucket = 'your-bucket-name'
prefix = 'deepar/deepar-bikerental' # change to your desired S3 prefix

# This structure allows multiple training and test files for model development and testing
if with_categories:
    s3_data_path = "{}/{}/data_with_categories".format(bucket, prefix)
else:
    s3_data_path = "{}/{}/data".format(bucket, prefix)
    

s3_output_path = "{}/{}/output".format(bucket, prefix)

In [4]:
s3_data_path,s3_output_path

('sagemaker-eu-west-1-304527814092/deepar/deepar-bikerental/data',
 'sagemaker-eu-west-1-304527814092/deepar/deepar-bikerental/output')

In [5]:
def write_to_s3(filename, bucket, key):
    with open(filename,'rb') as f: # Read in binary mode
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [19]:
# Upload one or more training files and test files to S3
if with_categories:
    write_to_s3('train_with_categories.json',bucket,'deepar/deepar-bikerental/data_with_categories/train/train_with_categories.json')
    write_to_s3('test_with_categories.json',bucket,'deepar/deepar-bikerental/data_with_categories/test/test_with_categories.json')
else:
    write_to_s3('train_bike.json',bucket,'deepar/deepar-bikerental/data/train/train_bike.json')
    write_to_s3('test_bike.json',bucket,'deepar/deepar-bikerental/data/test/test_bike.json')
    
    

In [20]:
sess = sagemaker.Session()
role = get_execution_role()

In [21]:
container = sagemaker.image_uris.retrieve("forecasting-deepar",sess.boto_region_name)

print (f'Using DeepAR Container {container}')

Using DeepAR Container 224300973850.dkr.ecr.eu-west-1.amazonaws.com/forecasting-deepar:1


In [22]:
container

'224300973850.dkr.ecr.eu-west-1.amazonaws.com/forecasting-deepar:1'

In [23]:
freq='H' 

prediction_length = 288

context_length = 288

In [24]:
estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path="s3://" + s3_output_path,
    sagemaker_session=sess,
    base_job_name=base_job_name)

In [25]:
freq, context_length, prediction_length

('H', 288, 288)

In [26]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "288",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "cardinality" : "auto" if with_categories else ''
}

In [27]:
hyperparameters

{'time_freq': 'H',
 'epochs': '400',
 'early_stopping_patience': '288',
 'mini_batch_size': '64',
 'learning_rate': '5E-4',
 'context_length': '288',
 'prediction_length': '288',
 'cardinality': ''}

In [28]:
estimator.set_hyperparameters(**hyperparameters)

In [29]:
data_channels = {
    "train": "s3://{}/train/".format(s3_data_path),
    "test": "s3://{}/test/".format(s3_data_path)
}

In [32]:
data_channels

{'train': 's3://sagemaker-eu-west-1-304527814092/deepar/deepar-bikerental/data/train/',
 'test': 's3://sagemaker-eu-west-1-304527814092/deepar/deepar-bikerental/data/test/'}

In [31]:
estimator.fit(inputs=data_channels)

2021-04-04 08:15:24 Starting - Starting the training job...
2021-04-04 08:15:49 Starting - Launching requested ML instancesProfilerReport-1617524124: InProgress
......
2021-04-04 08:16:50 Starting - Preparing the instances for training......
2021-04-04 08:17:52 Downloading - Downloading input data
2021-04-04 08:17:52 Training - Downloading the training image...
2021-04-04 08:18:10 Training - Training image download completed. Training in progress.Arguments: train
[04/04/2021 08:18:10 INFO 139780359198336] Reading default configuration from /opt/amazon/lib/python3.6/site-packages/algorithm/resources/default-input.json: {'_kvstore': 'auto', '_num_gpus': 'auto', '_num_kv_servers': 'auto', '_tuning_objective_metric': '', 'cardinality': 'auto', 'dropout_rate': '0.10', 'early_stopping_patience': '', 'embedding_dimension': '10', 'learning_rate': '0.001', 'likelihood': 'student-t', 'mini_batch_size': '128', 'num_cells': '40', 'num_dynamic_feat': 'auto', 'num_eval_samples': '100', 'num_layers':

[04/04/2021 08:19:08 INFO 139780359198336] Epoch[2] Batch[5] avg_epoch_loss=3.460503
[04/04/2021 08:19:08 INFO 139780359198336] #quality_metric: host=algo-1, epoch=2, batch=5 train loss <loss>=3.4605026245117188
[04/04/2021 08:19:08 INFO 139780359198336] Epoch[2] Batch [5]#011Speed: 54.27 samples/sec#011loss=3.460503
[04/04/2021 08:19:14 INFO 139780359198336] Epoch[2] Batch[10] avg_epoch_loss=3.526406
[04/04/2021 08:19:14 INFO 139780359198336] #quality_metric: host=algo-1, epoch=2, batch=10 train loss <loss>=3.605489206314087
[04/04/2021 08:19:14 INFO 139780359198336] Epoch[2] Batch [10]#011Speed: 61.57 samples/sec#011loss=3.605489
[04/04/2021 08:19:14 INFO 139780359198336] processed a total of 650 examples
#metrics {"StartTime": 1617524338.5787141, "EndTime": 1617524354.0546103, "Dimensions": {"Algorithm": "AWS/DeepAR", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 15475.832223892212, "count": 1, "min": 15475.832223892212, "max": 15475.832223892212}}}


[04/04/2021 08:20:14 INFO 139780359198336] Epoch[7] Batch[0] avg_epoch_loss=3.083492
[04/04/2021 08:20:14 INFO 139780359198336] #quality_metric: host=algo-1, epoch=7, batch=0 train loss <loss>=3.083491563796997
[04/04/2021 08:20:20 INFO 139780359198336] Epoch[7] Batch[5] avg_epoch_loss=3.007710
[04/04/2021 08:20:20 INFO 139780359198336] #quality_metric: host=algo-1, epoch=7, batch=5 train loss <loss>=3.007709503173828
[04/04/2021 08:20:20 INFO 139780359198336] Epoch[7] Batch [5]#011Speed: 62.84 samples/sec#011loss=3.007710
[04/04/2021 08:20:25 INFO 139780359198336] Epoch[7] Batch[10] avg_epoch_loss=3.046127
[04/04/2021 08:20:25 INFO 139780359198336] #quality_metric: host=algo-1, epoch=7, batch=10 train loss <loss>=3.0922286033630373
[04/04/2021 08:20:25 INFO 139780359198336] Epoch[7] Batch [10]#011Speed: 59.66 samples/sec#011loss=3.092229
[04/04/2021 08:20:25 INFO 139780359198336] processed a total of 668 examples
#metrics {"StartTime": 1617524410.837335, "EndTime": 1617524425.389329, 

[04/04/2021 08:21:31 INFO 139780359198336] Epoch[12] Batch[5] avg_epoch_loss=2.874444
[04/04/2021 08:21:31 INFO 139780359198336] #quality_metric: host=algo-1, epoch=12, batch=5 train loss <loss>=2.8744444449742637
[04/04/2021 08:21:31 INFO 139780359198336] Epoch[12] Batch [5]#011Speed: 62.67 samples/sec#011loss=2.874444
[04/04/2021 08:21:37 INFO 139780359198336] Epoch[12] Batch[10] avg_epoch_loss=2.804211
[04/04/2021 08:21:37 INFO 139780359198336] #quality_metric: host=algo-1, epoch=12, batch=10 train loss <loss>=2.71993145942688
[04/04/2021 08:21:37 INFO 139780359198336] Epoch[12] Batch [10]#011Speed: 59.74 samples/sec#011loss=2.719931
[04/04/2021 08:21:37 INFO 139780359198336] processed a total of 647 examples
#metrics {"StartTime": 1617524482.4364154, "EndTime": 1617524497.0563815, "Dimensions": {"Algorithm": "AWS/DeepAR", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 14619.868278503418, "count": 1, "min": 14619.868278503418, "max": 14619.86827850341

[04/04/2021 08:22:38 INFO 139780359198336] Epoch[17] Batch[0] avg_epoch_loss=2.806967
[04/04/2021 08:22:38 INFO 139780359198336] #quality_metric: host=algo-1, epoch=17, batch=0 train loss <loss>=2.80696702003479
[04/04/2021 08:22:44 INFO 139780359198336] Epoch[17] Batch[5] avg_epoch_loss=2.802652
[04/04/2021 08:22:44 INFO 139780359198336] #quality_metric: host=algo-1, epoch=17, batch=5 train loss <loss>=2.802651802698771
[04/04/2021 08:22:44 INFO 139780359198336] Epoch[17] Batch [5]#011Speed: 62.80 samples/sec#011loss=2.802652
[04/04/2021 08:22:48 INFO 139780359198336] processed a total of 635 examples
#metrics {"StartTime": 1617524554.6945581, "EndTime": 1617524568.192489, "Dimensions": {"Algorithm": "AWS/DeepAR", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 13497.579574584961, "count": 1, "min": 13497.579574584961, "max": 13497.579574584961}}}

[04/04/2021 08:22:48 INFO 139780359198336] #throughput_metric: host=algo-1, train throughput=47.04512699271

...

[04/04/2021 09:48:29 INFO 139780359198336] Epoch[378] Batch[5] avg_epoch_loss=2.427793
[04/04/2021 09:48:29 INFO 139780359198336] #quality_metric: host=algo-1, epoch=378, batch=5 train loss <loss>=2.427792946497599
[04/04/2021 09:48:29 INFO 139780359198336] Epoch[378] Batch [5]#011Speed: 63.04 samples/sec#011loss=2.427793
[04/04/2021 09:48:33 INFO 139780359198336] processed a total of 614 examples
#metrics {"StartTime": 1617529700.4239316, "EndTime": 1617529713.901887, "Dimensions": {"Algorithm": "AWS/DeepAR", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 13477.628946304321, "count": 1, "min": 13477.628946304321, "max": 13477.628946304321}}}

[04/04/2021 09:48:33 INFO 139780359198336] #throughput_metric: host=algo-1, train throughput=45.556554761193716 records/second
[04/04/2021 09:48:33 INFO 139780359198336] #progress_metric: host=algo-1, completed 94.75 % of epochs
[04/04/2021 09:48:33 INFO 139780359198336] #quality_metric: host=algo-1, epoch=378, tra

[04/04/2021 09:49:47 INFO 139780359198336] Epoch[384] Batch[0] avg_epoch_loss=2.465906
[04/04/2021 09:49:47 INFO 139780359198336] #quality_metric: host=algo-1, epoch=384, batch=0 train loss <loss>=2.4659059047698975
[04/04/2021 09:49:52 INFO 139780359198336] Epoch[384] Batch[5] avg_epoch_loss=2.509413
[04/04/2021 09:49:52 INFO 139780359198336] #quality_metric: host=algo-1, epoch=384, batch=5 train loss <loss>=2.509412964185079
[04/04/2021 09:49:52 INFO 139780359198336] Epoch[384] Batch [5]#011Speed: 60.66 samples/sec#011loss=2.509413
[04/04/2021 09:49:58 INFO 139780359198336] Epoch[384] Batch[10] avg_epoch_loss=2.508626
[04/04/2021 09:49:58 INFO 139780359198336] #quality_metric: host=algo-1, epoch=384, batch=10 train loss <loss>=2.507682180404663
[04/04/2021 09:49:58 INFO 139780359198336] Epoch[384] Batch [10]#011Speed: 59.10 samples/sec#011loss=2.507682
[04/04/2021 09:49:58 INFO 139780359198336] processed a total of 688 examples
#metrics {"StartTime": 1617529783.291966, "EndTime": 161

[04/04/2021 09:51:10 INFO 139780359198336] processed a total of 616 examples
#metrics {"StartTime": 1617529855.5452306, "EndTime": 1617529870.0864136, "Dimensions": {"Algorithm": "AWS/DeepAR", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 14540.778160095215, "count": 1, "min": 14540.778160095215, "max": 14540.778160095215}}}

[04/04/2021 09:51:10 INFO 139780359198336] #throughput_metric: host=algo-1, train throughput=42.363340732828426 records/second
[04/04/2021 09:51:10 INFO 139780359198336] #progress_metric: host=algo-1, completed 97.5 % of epochs
[04/04/2021 09:51:10 INFO 139780359198336] #quality_metric: host=algo-1, epoch=389, train loss <loss>=2.450956630706787
[04/04/2021 09:51:10 INFO 139780359198336] loss did not improve
[04/04/2021 09:51:14 INFO 139780359198336] Epoch[390] Batch[0] avg_epoch_loss=2.554639
[04/04/2021 09:51:14 INFO 139780359198336] #quality_metric: host=algo-1, epoch=390, batch=0 train loss <loss>=2.554638624191284
[04/04/2021 

[04/04/2021 09:52:31 INFO 139780359198336] Epoch[395] Batch[5] avg_epoch_loss=2.432020
[04/04/2021 09:52:31 INFO 139780359198336] #quality_metric: host=algo-1, epoch=395, batch=5 train loss <loss>=2.4320202668507895
[04/04/2021 09:52:31 INFO 139780359198336] Epoch[395] Batch [5]#011Speed: 63.19 samples/sec#011loss=2.432020
[04/04/2021 09:52:36 INFO 139780359198336] Epoch[395] Batch[10] avg_epoch_loss=2.409468
[04/04/2021 09:52:36 INFO 139780359198336] #quality_metric: host=algo-1, epoch=395, batch=10 train loss <loss>=2.382405424118042
[04/04/2021 09:52:36 INFO 139780359198336] Epoch[395] Batch [10]#011Speed: 59.52 samples/sec#011loss=2.382405
[04/04/2021 09:52:36 INFO 139780359198336] processed a total of 685 examples
#metrics {"StartTime": 1617529942.1292264, "EndTime": 1617529956.5949373, "Dimensions": {"Algorithm": "AWS/DeepAR", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 14465.210676193237, "count": 1, "min": 14465.210676193237, "max": 14465.2106

#metrics {"StartTime": 1617530010.8708496, "EndTime": 1617530025.5226464, "Dimensions": {"Algorithm": "AWS/DeepAR", "Host": "algo-1", "Operation": "training"}, "Metrics": {"get_graph.time": {"sum": 14650.559902191162, "count": 1, "min": 14650.559902191162, "max": 14650.559902191162}}}

[04/04/2021 09:53:46 INFO 139780359198336] Number of GPUs being used: 0
#metrics {"StartTime": 1617530025.5227408, "EndTime": 1617530026.6047297, "Dimensions": {"Algorithm": "AWS/DeepAR", "Host": "algo-1", "Operation": "training"}, "Metrics": {"finalize.time": {"sum": 15732.687711715698, "count": 1, "min": 15732.687711715698, "max": 15732.687711715698}}}

[04/04/2021 09:53:46 INFO 139780359198336] Serializing to /opt/ml/model/model_algo-1
[04/04/2021 09:53:46 INFO 139780359198336] Saved checkpoint to "/opt/ml/model/model_algo-1-0000.params"
#metrics {"StartTime": 1617530026.6048021, "EndTime": 1617530026.7355595, "Dimensions": {"Algorithm": "AWS/DeepAR", "Host": "algo-1", "Operation": "training"}, "Metri

In [34]:
job_name = estimator.latest_training_job.name
print ('job name: {0}'.format(job_name))

job name: deepar-bikerental-no-categories-2021-04-04-08-15-24-671


In [35]:
endpoint_name = sess.endpoint_from_job(
    job_name=job_name,
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    image_uri=container,
    role=role)

---------------!

In [36]:

print ('endpoint name: {0}'.format(endpoint_name))

endpoint name: deepar-bikerental-no-categories-2021-04-04-08-15-24-671
